# Optimization the model parameters

Now that we have a model and data it's time to train, validate and test our model by optimizing its parameters on our data. Training a model is an iterative process, in each iteraction (called an epoch) the model makes a guess about the ouput, calculates the error in its guess (loss), collects the derivatives of the error with respect to its parameters (as we saw in the module), and opimizes these parameters using gradient descent. For a more detailed walkthrough of his process, check out this video on [backpropagation from 3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8).

In [1]:
import torch 
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torch import nn 
from torch.utils.data import DataLoader


In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    transform=ToTensor(),
    download=True
)

C:\Users\duong\AppData\Local\Programs\Python\Python38\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
train_data_loader = DataLoader(training_data, batch_size=64)
test_data_loader = DataLoader(test_data, batch_size=64)

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
model = NeuralNetwork()

## Setting hyperparameters

Hyperparameters are adjustable parameters that let you control the model optimization process. Different hyperparameter values can impact model training and convergence rate.

We define the following hyperparameters for training:
* **Number of Epochs** - the number of times to iterate over the dataset. 
* **Batch Size** - the number of data samples seen by the model in each epoch. 
* **Learning Rate** - How much to update models parameters at each epoch. Smaller values yield show learning speed, while large values may result in unpreditable behavior during training. 

In [7]:
learning_rate = 1e-3
batch_size = 64 
epochs = 5

## Add an optimization loop

Once we set our hyperparameters, we can then train and optimize our model with an optimization loop. Each iteraction of the optimization loop is called an **epoch**.

Each epoch consists of two main parts:
* **The Train Loop** - iterate over the training dataset and try to convergence to optimal parameters. 
* **The Validation/Test Loop** - iterate over the test dataset to check if model performance is improving. 

Let's briefly familiarize ourselves with some of the concepts used in the training loop. Jump ahead to see the **full-impl-label** of the optimization loop

## Add a loss functino

When presented with some training data, our untrained network is likely not to given the correct answer. **Lost function** measures the degree of dissimilarity of obtained result to the target value, and it is the loss function that we want to minimize during training. To caculate the loss we make a prediction using the inputs of our given data sample and compare it against the true data label value.

Common loss functions include **nn.MSELoss** for regression tasks, and **nn.NLLLoss** (Negative Log Likelihood) for classification. **nn.CrossEntropyLoss** combines **nn.LogSoftmax** and nn.**NLLLoss**.

We pass our model's output logits to nn.CrossEntropyLoss, which will normalize the logits and compute the prediction error.

In [8]:
loss_fn = nn.CrossEntropyLoss()

## Optimization pass

Optimization is the process of adjusting model parameters  to reduce model error in each training step. **Optimization algorithm** define how this process is performed.


In [10]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


Inside the training loop, optimization happens in three steps:
* Call **optimizer.zero_grad()** to reset the gradient of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteraction.
* Back-propagate the prediction loss with a call to loss.backwards(). PyTorch deposits the gradients of the loss w.r.t each parameter.
* Once we have our gradients, we call **optimizer.step()** to adjust the parameters by the gradients collected in the backward pass.



In [11]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        #Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch* len(X)
            print(f'loss: {loss:>7f}  [{current:>5f}/{size:>5f}]')

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y, in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size

    print(f'Test error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')

In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_data_loader, model, loss_fn, optimizer)
    test_loop(test_data_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.304427  [0.000000/60000.000000]
loss: 2.300269  [6400.000000/60000.000000]
loss: 2.283340  [12800.000000/60000.000000]
loss: 2.285034  [19200.000000/60000.000000]
loss: 2.269241  [25600.000000/60000.000000]
loss: 2.253967  [32000.000000/60000.000000]
loss: 2.250596  [38400.000000/60000.000000]
loss: 2.220184  [44800.000000/60000.000000]
loss: 2.231462  [51200.000000/60000.000000]
loss: 2.240261  [57600.000000/60000.000000]


TypeError: test_loop() missing 1 required positional argument: 'optimizer'